<img src="https://drive.google.com/uc?id=1-cL5eOpEsbuIEkvwW2KnpXC12-PAbamr" style="Width:1000px">

# Preprocessor Pipeline
### Back to Golden Plains Roadside Biodiversity

As our first exercise we will return to the  <a href='https://data.gov.au/data/dataset/golden-plains-roadside-biodiversity'>Golden Plains Shire (Australia) dataset</a>.<br>
![plain](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6b/Mount_Conner%2C_August_2003.jpg/375px-Mount_Conner%2C_August_2003.jpg)
<br>

🎯 As last time, the exercise will consist of the data preparation and feature selection techniques you have learnt: we will again try to predict via linear regression the `RCACScore`. But this time, we will do it using `Pipelines`, and we don't really care that much about the final score (the goal is to demonstrate how pipelines work.

The goal is to demonstrate how helpful pipelines are in making our code cleaner, and avoid repetition.

👇 Load the data into this notebook as a pandas dataframe named `df`, and display its first 5 rows.

In [ ]:
from nbta.utils import download_data
download_data(id='1SUxBmDZF6fsu3ndrgrNI7aI2B2BEr59S')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('raw_data/biodiversity.csv')
df.head(5)

# Remove Duplicates and Hold Out

Removing duplicates still **needs to be done before training a pipeline**. Why? Because if you don't do it now, once you train-test-split (hold out method) you might have data leakage. Do the following:
1. Remove the duplicates
2. Create the target `y` vector (`RCACScore`) and feature `X` matrix
3. Using a 30% split for the test set, create the `X_train`, `X_test`, `y_train`, `y_test`

In [ ]:
# The code below ensures that all transformers return pandas dataframe - makes life easier!

from sklearn import set_config
set_config(transform_output="pandas")

In [ ]:
# Find the number of duplicates
df.duplicated().sum()

In [ ]:
# Drop them in place and check
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
from sklearn.model_selection import train_test_split

y = df.RCACScore
X = df.drop(columns=['RCACScore'])

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3, random_state=42)

# Creating our numeric pipeline

Let's start by taking care of the numeric values, and do this in a pipeline. The steps we will want to implements are the following:
1. Impute values with a `SimpleImputer`, with the default strategy
2. Scale the values using a `StandardScaler`

Create a pipeline that does just that! Call it `num_pipe`. Then, fit it to the **numeric columns of your** `X_train`.

In [ ]:
from sklearn import set_config

set_config(display='diagram')

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipe = make_pipeline(SimpleImputer(), StandardScaler())

In [ ]:
num_cols = X_train.select_dtypes(include=np.number).columns
num_pipe.fit(X_train[num_cols])

# Applying your pipline

Create two new dataframes named `X_train_prep` and `X_test_prep`,  containing respectively the values of `X_train` and `X_test` transformed by your pipeline.

**Tip:** Note that the transform function of a pipeline returns a `NumPy array` by default. So, to obtain a nice looking datadframe, you will need to create it from the return numpy array and the columns of the original `X_train` and `X_test` dataframes.

In [ ]:
X_train_prep = pd.DataFrame(num_pipe.transform(X_train[num_cols]), columns=num_cols)
X_train_prep

In [ ]:
X_test_prep = pd.DataFrame(num_pipe.transform(X_test[num_cols]), columns=num_cols)

In [ ]:
X_test_prep

<details>
    <summary>💡 Conclusions</summary>
<br>
ℹ️ Hopefully you can see with this simple example that <code>Pipeline</code>s make your life much easier! <br> 
    Once you have fitted your pipeline to the <code>X_train</code>, you can <code>transform</code> the <code>X_train</code>, <code>X_test</code>, or indeed any new data you need to predict!
    
</details>

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('num_pipe',
                         X_test = X_test_prep
)

result.write()
print(result.check())

# Categorical Data Pipeline

Let's now build a new `pipeline` and name it `cat_pipe` for **Categorical Pipeline**. We will deal with the categorical data by doing the following:
- Apply a `SimpleImputer()` with the `most_frequent` strategy
- Apply a `OneHotEncoder()` with the `handle_unknown="ignore"` to the categorical data

Then, transform the `X_train` and `X_test` categorical data into a `X_train_cat_prep` and `X_test_cat_prep` in the same way as you trensformed the numerical data above.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = df.select_dtypes(exclude = np.number).columns

cat_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(handle_unknown='ignore', sparse_output=False))

In [ ]:
cat_pipe.fit(X_train[cat_cols])

X_train_cat_prep = cat_pipe.transform(X_train[cat_cols])
X_test_cat_prep = cat_pipe.transform(X_test[cat_cols])

In [ ]:
X_train_cat_prep.shape

In [ ]:
X_train_cat_prep

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('cat_pipe',
                         X_test = X_test_cat_prep)
result.write()
print(result.check())

# Building a single processing Pipeline

Ok, the process above was already saving us a lot of time. But now, we are going to take this to the next level by creating a `ColumnTransformer` that groups both `pipelines`together! Name your `ColumnTransformer` as `preproc_pipe` (**preprocessing pipeline**). If in doubt, consult the documentation!

Once your pipeline is created, you can `fit` it on the `X_train` and then `transform` the `X_train` and the `X_test` into `X_train_preproc` and `X_test_preproc`: you will see how easy it makes your code.

In [ ]:
from sklearn.compose import ColumnTransformer

preproc_pipe = ColumnTransformer([
    ("num_prep", num_pipe, X_train.select_dtypes(include=np.number).columns),
    ("cat_prep", cat_pipe, X_train.select_dtypes(exclude=np.number).columns)])

In [ ]:
preproc_pipe.fit(X_train)

In [ ]:
X_train_preproc = preproc_pipe.transform(X_train)
X_test_preproc = preproc_pipe.transform(X_test)
X_train_preproc

### ☑️ Test your code

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('preproc_pipe',
                         X_test = X_test_preproc
)

result.write()
print(result.check())

# Adding models into pipelines

The true power of the `pipeline` class is that it can also handle full `sklearn` models! So, all we need to do now is create a new `pipeline`, let's call it `linear_model`, and this will contain the `preproc_pipe` and a `LinearRegression` model. Then, you can directly fit it to the `X_train` and `y_train`. Onces this is fitted, you can estimate the `y_pred` from the `X_test`, and save the `root mean squared error` of this model into a new variable, `score`.



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error

linear_model = Pipeline([
    ('preproc',preproc_pipe),
('model', LinearRegression())])

In [ ]:
linear_model.fit(X_train, y_train)

In [ ]:
y_pred = linear_model.predict(X_test)

In [ ]:
score = np.sqrt(mean_squared_error(y_test, y_pred))
score

# 🏁 Finished!

Well done! <span style="color:teal">**Push your exercise to GitHub**</span>, and move on to the next one.